In [83]:
import os

In [84]:
# !pip install ccxt
# !pip install pandas

1. Изучить документацию RESTful API Binance (https://binance-docs.github.io/apidocs/spot/en/) и написать скрипт на Python с использованием библиотеки Requests, чтобы получить данные о котировках криптовалют. Сохранить полученные данные в формате CSV.

In [85]:
import ccxt
import pandas as pd

def grab_data(source, symbol, timeframe, start_timestamp, path):
    # Get OHLCV data from Binance API
    ohlcv_data = source.fetch_ohlcv(symbol, timeframe, int(start_timestamp))

    # Convert OHLCV data to Pandas DataFrame
    ohlcv_df = pd.DataFrame(ohlcv_data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])

    # Convert timestamp from Unix to datetime format
    ohlcv_df['timestamp'] = pd.to_datetime(ohlcv_df['timestamp'], unit = 'ms')

    # Set timestamp as index of the DataFrame
    ohlcv_df.set_index('timestamp', inplace=True)

    # Save DataFrame to CSV file
    if not os.path.exists(path):
        os.makedirs(path)
    ohlcv_df.to_csv(f'{path}/{symbol.lower().replace("/", "_")}_historical_data.csv', index=True)
    
    # return ohlcv_df

def grab_and_save_data(exchange, symbols, timeframe, start_date, path):

    # Initialize Binance exchange using ccxt
    source = getattr (ccxt, exchange) ({'enableRateLimit': True})

    # Convert start and end dates to Unix timestamp
    start_timestamp = pd.Timestamp(start_date).timestamp() * 1000

    if isinstance(symbols, list):
        for symbol in symbols:
            grab_data(source, symbol, timeframe, start_timestamp, path)
    else:
        grab_data(source, symbol, timeframe, start_timestamp, path)

start_date = '2021-01-01'
timeframe = '1d'
symbols = ['BTC/USDT','ATOM/USDT']
exchange = 'binance'
path = 'data/'

grab_and_save_data(exchange, symbols, timeframe, start_date, path)


2. Загрузить полученный CSV-файл с данными в Python с использованием библиотеки Pandas и выполнить предварительную обработку данных (например, удаление пропусков, преобразование даты и времени, масштабирование).

In [86]:
def data_preparing(path, symbol):
    
    # Read CSV file
    ohlcv_df = pd.read_csv(path+symbol.lower().replace("/", "_")+"_historical_data.csv")
    
    # Convert timetamp to Pandas datetime object
    ohlcv_df['timestamp'] = pd.to_datetime(ohlcv_df['timestamp'], format='%Y-%m-%d')
    
    # Set datetie as index
    ohlcv_df.set_index('timestamp', inplace=True)
    
    # Scaling data with Pandas and Numpy
    # Min-Max Normalization
    ohlcv_df_norm = (ohlcv_df-ohlcv_df.min())/(ohlcv_df.max()-ohlcv_df.min()) 
    ohlcv_df_norm

    return ohlcv_df_norm

ohlcv_df_norm = data_preparing(path, 'BTC/USDT')

ohlcv_df_norm

,open,high,low,close,volume
timestamp,,,,,
2021-01-01,0.000000,0.000000,0.048696,0.007843,0.113361
2021-01-02,0.010571,0.093909,0.056842,0.081790,0.337295
2021-01-03,0.084265,0.131424,0.133165,0.103135,0.310603
2021-01-04,0.105601,0.101523,0.036182,0.076864,0.369510
2021-01-05,0.079429,0.120812,0.080967,0.127800,0.296107
...,...,...,...,...,...
2022-05-11,0.054232,0.065040,0.027453,0.001927,0.564947
2022-05-12,0.004671,0.016320,0.000000,0.000000,0.557396
2022-05-13,0.002749,0.037649,0.051912,0.006684,0.242413


3. С использованием библиотеки NumPy, разделить данные на обучающую и тестовую выборки.

In [87]:

# Dividing the data into training and test samples
train_data = ohlcv_df_norm[:int(0.8*len(ohlcv_df_norm))]
test_data = ohlcv_df_norm[int(0.8*len(ohlcv_df_norm)):]

4. Создать и обучить LSTM-модель с использованием библиотеки Keras или TensorFlow на основе подготовленных данных. Подобрать архитектуру и гиперпараметры модели, чтобы достичь наилучшего качества прогнозирования.

5. Оценить качество модели на тестовой выборке, используя подходящие метрики, такие как среднеквадратичная ошибка (MSE) или средняя абсолютная ошибка (MAE).

6. Визуализировать предсказания модели для тестовой выборки на графике с помощью библиотеки Matplotlib или Plotly. Вывести график с предсказанием на час вперед от последних полученных данных.